In [4]:
import os
import pandas as pd
from src.experiments.consts import ExperimentType

path = os.getcwd()
survey_path = "{0}/survey_gamification".format(os.getcwd())
results = f'{os.getcwd()}/results_30'
fee_path = f'{results}/FEATURE_ENGINEERING'
ete_path = f'{results}/END_TO_END'

In [10]:
from src.data_prepare import prepare_data

prepare_data('/home/mateus/Documents/school/tcc/survey_gamification')

remove survey measurements
remove unused files
merging files at: /home/mateus/Documents/school/tcc/survey_gamification
  SUBJECT: 21
    file: samsung_ppg.csv - 0.06"
    file: empatica_ppg.csv - 0.1"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 4.76"
  SUBJECT: 13
    file: samsung_ppg.csv - 0.07"
    file: empatica_ppg.csv - 0.19"
    file: empatica_eda.csv - 0.02"
    file: muse_eeg.csv - 3.56"
  SUBJECT: 17
    file: samsung_ppg.csv - 0.04"
    file: empatica_ppg.csv - 0.07"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 3.14"
  SUBJECT: 20
    file: samsung_ppg.csv - 0.02"
    file: empatica_ppg.csv - 0.05"
    file: empatica_eda.csv - 0.01"


/home/mateus/Documents/school/tcc/code/src/data_prepare.py:50: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path)


    file: muse_eeg.csv - 0.96"
  SUBJECT: 14
    file: samsung_ppg.csv - 0.03"
    file: empatica_ppg.csv - 0.07"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 1.67"
  SUBJECT: 15
    file: samsung_ppg.csv - 0.03"
    file: empatica_ppg.csv - 0.07"
    file: empatica_eda.csv - 0.01"


/home/mateus/Documents/school/tcc/code/src/data_prepare.py:50: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path)


    file: muse_eeg.csv - 3.0"
  SUBJECT: 16
    file: samsung_ppg.csv - 0.03"
    file: empatica_ppg.csv - 0.07"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 2.94"
  SUBJECT: 18
    file: samsung_ppg.csv - 0.03"
    file: empatica_ppg.csv - 0.08"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 3.32"
  SUBJECT: 24
    file: samsung_ppg.csv - 0.04"
    file: empatica_ppg.csv - 0.08"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 2.32"
  SUBJECT: 11
    file: samsung_ppg.csv - 0.03"
    file: empatica_ppg.csv - 0.07"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 1.41"
  SUBJECT: 22
    file: samsung_ppg.csv - 0.03"
    file: empatica_ppg.csv - 0.07"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 3.17"
  SUBJECT: 23
    file: samsung_ppg.csv - 0.03"
    file: empatica_ppg.csv - 0.07"
    file: empatica_eda.csv - 0.01"
    file: muse_eeg.csv - 3.38"
  SUBJECT: 12
    file: samsung_ppg.csv - 0.04"
    file: empatica_ppg.csv -

In [13]:
def ft(seconds):
  return f"{seconds // 60}\'{seconds % 60}\""

stat = pd.DataFrame(columns=["volunteer", "sms_na", "sms_total", "sms_recording", "emp_na", "emp_total", "emp_recording", "eda_na", "eda_total", "eda_recording", "eeg_na", "eeg_total", "eeg_recording"])
vols = os.listdir(survey_path)
vols.sort()
for v in vols:
  metrics = {"volunteer": v,
             'sms_min': 0, 'sms_max': 0, 'sms_mean': 0, 'sms_std': 0,
             'emp_min': 0, 'emp_max': 0, 'emp_mean': 0, 'emp_std': 0,
             'eda_min': 0, 'eda_max': 0, 'eda_mean': 0, 'eds_std': 0,
             'eeg_min': 0, 'eeg_max': 0, 'eeg_mean': 0, 'eeg_std': 0,
             'sms_ratio': 0,'emp_ratio': 0,'eda_ratio': 0,'eeg_ratio': 0, 'sms_t': '', 'emp_t': '', 'eda_t': '', 'eeg_t': '', "sms_na": 0, "sms_total": 0, "sms_recording": 0, "emp_na": 0, "emp_total": 0, "emp_recording": 0, "eda_na": 0, "eda_total": 0, "eda_recording": 0, "eeg_na": 0, "eeg_total": 0, "eeg_recording": 0}
  samsung = pd.read_csv(os.path.join(survey_path, v, "samsung_ppg.csv"))
  emp = pd.read_csv(os.path.join(survey_path, v, "empatica_ppg.csv"))
  eda = pd.read_csv(os.path.join(survey_path, v, "empatica_eda.csv"))
  eeg = pd.read_csv(os.path.join(survey_path, v, "muse_eeg.csv"))
  # print(f"Volunteer {v}")

  total = samsung.shape[0]
  metrics["sms_total"] = total
  na = samsung.isnull().any(axis=1).sum()
  metrics["sms_na"] = na
  seconds = total // 25
  metrics["sms_recording"] = seconds
  metrics["sms_t"] = ft(seconds)
  cl_s = len(samsung.where(samsung['y'] == 1).dropna())
  bl_s = len(samsung.where(samsung['y'] == 0).dropna())
  metrics['sms_min'] = round(samsung['ppg'].min(), 3).__str__()
  metrics['sms_max'] = round(samsung['ppg'].max(), 2).__str__()
  metrics['sms_mean'] = round(samsung['ppg'].mean(), 3).__str__()
  metrics['sms_std'] = round(samsung['ppg'].std(), 3).__str__()
  s_ratio = round(cl_s / bl_s, 3)
  metrics['sms_ratio'] = s_ratio
  # print(f"  PPG| na: {na} | total: {total} | ratio: {s_ratio} | {seconds // 60}\' {seconds % 60}\"")

  total = emp.shape[0]
  metrics["emp_total"] = total
  na = emp.isnull().any(axis=1).sum()
  metrics["emp_na"] = na
  cl = len(emp.where(emp['y'] == 1).dropna())
  bl = len(emp.where(emp['y'] == 0).dropna())
  seconds = total // 64
  metrics["emp_t"] = ft(seconds)
  metrics["emp_recording"] = seconds
  emp_ratio = round(cl / bl, 3)
  metrics['emp_ratio'] = emp_ratio
  metrics['emp_min'] = round(emp['ppg'].min(), 3).__str__()
  metrics['emp_max'] = round(emp['ppg'].max(), 3).__str__()
  metrics['emp_mean'] = round(emp['ppg'].mean(), 3).__str__()
  metrics['emp_std'] = round(emp['ppg'].std(), 3).__str__()
  # print(f"  EMP| na: {na} | total: {total} | ratio: {emp_ratio} | {seconds // 60}\' {seconds % 60}\"")

  total = eda.shape[0]
  metrics["eda_total"] = total
  na = eda.isnull().any(axis=1).sum()
  cl = len(eda.where(eda['y'] == 1).dropna())
  bl = len(eda.where(eda['y'] == 0).dropna())
  metrics["eda_na"] = na
  seconds = total // 4
  metrics["eda_t"] = ft(seconds)
  metrics["eda_recording"] = seconds
  eda_ratio = round(cl / bl, 3)
  metrics['eda_ratio'] = eda_ratio
  metrics['eda_min'] = round(eda['eda'].min(), 3).__str__()
  metrics['eda_max'] = round(eda['eda'].max(), 3).__str__()
  metrics['eda_mean'] = round(eda['eda'].mean(), 3).__str__()
  metrics['eda_std'] = round(eda['eda'].std(), 3).__str__()
  # print(f"  EDA| na: {na} | total: {total} | ratio: {eda_ratio} | {seconds // 60}\' {seconds % 60}\"")

  total = eeg.shape[0]
  metrics["eeg_total"] = total
  na = eeg.isnull().any(axis=1).sum()
  metrics["eeg_na"] = na
  cl = len(eeg.where(eeg['y'] == 1).dropna())
  bl = len(eeg.where(eeg['y'] == 0).dropna())
  seconds = total // 256
  metrics["eeg_t"] = ft(seconds)
  metrics["eeg_recording"] = seconds
  eeg_ratio = round(cl / (bl | 1), 3)
  metrics['eeg_ratio'] = eeg_ratio
  # print(f"  EEG| na: {na} | total: {total} | ratio: {eeg_ratio} | {seconds // 60}\' {seconds % 60}\"")

  latex = '''\
    \\midrule
\\parbox[t]{{2mm}}{{\\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{\\textbf {{Vol. {volunteer}}}}}}}}} &
      Samsung PPG  & {sms_t}    & {sms_ratio}    & {sms_total}  & {sms_na} & {sms_min} & {sms_max} & {sms_mean} & {sms_std}  \\\\
      & Empatica PPG  & {emp_t}    & {emp_ratio}    & {emp_total}  & {emp_na} & {emp_min} & {emp_max} & {emp_mean} & {emp_std} \\\\
      & Empatica EDA  & {eda_t}    & {eda_ratio}    & {eda_total}  & {eda_na}& {eda_min} & {eda_max} & {eda_mean} & {eda_std}  \\\\
      & Muse EEG  & {eeg_t}    & {eeg_ratio}    & {eeg_total}  & {eeg_na} & N/A & N/A & N/A & N/A \\\\
  '''.format(**metrics)
  print(latex)

  # row = pd.DataFrame(data=metrics, index=[0])
  # stat = pd.concat([stat, row], ignore_index=True)


    \midrule
\parbox[t]{2mm}{\multirow{4}{*}{\rotatebox[origin=c]{90}{\textbf {Vol. 11}}}} &
      Samsung PPG  & 13'38"    & 1.259    & 20474  & 0 & 1823074.0 & 2414563.0 & 2099593.878 & 127453.333  \\
      & Empatica PPG  & 13'35"    & 1.26    & 52174  & 0 & -239.73 & 231.78 & 0.002 & 46.877 \\
      & Empatica EDA  & 13'35"    & 1.26    & 3261  & 0& 0.231 & 0.569 & 0.472 & 0.064  \\
      & Muse EEG  & 6'29"    & 0.079    & 99730  & 72 & N/A & N/A & N/A & N/A \\
  
    \midrule
\parbox[t]{2mm}{\multirow{4}{*}{\rotatebox[origin=c]{90}{\textbf {Vol. 12}}}} &
      Samsung PPG  & 16'11"    & 1.69    & 24278  & 0 & 1315932.0 & 2520418.0 & 1854011.781 & 247407.574  \\
      & Empatica PPG  & 17'5"    & 1.695    & 65631  & 0 & -873.33 & 537.78 & 0.115 & 73.405 \\
      & Empatica EDA  & 17'4"    & 1.7    & 4099  & 0& 0.256 & 0.847 & 0.514 & 0.149  \\
      & Muse EEG  & 15'20"    & 1.553    & 235608  & 707 & N/A & N/A & N/A & N/A \\
  
    \midrule
\parbox[t]{2mm}{\multirow{4}{*}{\rotate

In [2]:
for row in stat.iterrows():
  print((row[1]["eeg_na"] / row[1]["eeg_total"]) * 100)

NameError: name 'stat' is not defined

In [10]:
class Setup():
  def __init__(self, type: ExperimentType):
    self.type = type
    self.classifiers = ["cnn", "fcn", "lstm", "resnet"] if ExperimentType.END_TO_END == type else ['knn', 'svm', 'xgb']
    results = f'{os.getcwd()}/results_30'
    fee_path = f'{results}/FEATURE_ENGINEERING'
    ete_path = f'{results}/END_TO_END'
    self.path = fee_path if ExperimentType.FEATURE_ENGINEERING == type else ete_path
    self.subject_ids = ['11', '12', '13', '14', '15', '16', '17', '18', '20', '21']

In [45]:
def metrics_for(setup: Setup, result={}):
  for device_signal in os.listdir(setup.path):
    if not device_signal in result:
      result[device_signal] = {}
    for classifier in setup.classifiers:
      if not classifier in result[device_signal]:
        result[device_signal][classifier] = {}
      for metrics in [f'{setup.path}/{device_signal}/test/{classifier}{id}_df_metrics.csv' for id in setup.subject_ids]:
        df = pd.read_csv(metrics).head()
        f1 = float(df['f1'].iloc[0])
        if not 'f1' in result[device_signal][classifier] or f1 > result[device_signal][classifier]["f1"]:
          result[device_signal][classifier]["f1"] = f1
          result[device_signal][classifier]["recall"] = float(df['recall'].iloc[0])
          result[device_signal][classifier]["precision"] = float(df['precision'].iloc[0])
          result[device_signal][classifier]["duration"] = float(df['duration'].iloc[0])
          result[device_signal][classifier]["model"] = metrics
  return result

In [46]:
data = metrics_for(Setup(type=ExperimentType.END_TO_END))
metrics_for(Setup(type=ExperimentType.FEATURE_ENGINEERING), result=data)

{'samsung_ppg': {'cnn': {'f1': 0.7482014388489209,
   'recall': 0.9285714285714286,
   'precision': 0.6265060240963856,
   'duration': 0.4055442810058594,
   'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/cnn16_df_metrics.csv'},
  'fcn': {'f1': 0.7883211678832117,
   'recall': 0.9642857142857144,
   'precision': 0.6666666666666666,
   'duration': 0.7080304622650146,
   'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/fcn15_df_metrics.csv'},
  'lstm': {'f1': 0.7567567567567568,
   'recall': 1.0,
   'precision': 0.6086956521739131,
   'duration': 4.65616250038147,
   'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/lstm11_df_metrics.csv'},
  'resnet': {'f1': 0.8115942028985508,
   'recall': 1.0,
   'precision': 0.6829268292682927,
   'duration': 2.4274494647979736,
   'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/resnet20_df_metr

In [47]:
data

{'samsung_ppg': {'cnn': {'f1': 0.7482014388489209,
   'recall': 0.9285714285714286,
   'precision': 0.6265060240963856,
   'duration': 0.4055442810058594,
   'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/cnn16_df_metrics.csv'},
  'fcn': {'f1': 0.7883211678832117,
   'recall': 0.9642857142857144,
   'precision': 0.6666666666666666,
   'duration': 0.7080304622650146,
   'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/fcn15_df_metrics.csv'},
  'lstm': {'f1': 0.7567567567567568,
   'recall': 1.0,
   'precision': 0.6086956521739131,
   'duration': 4.65616250038147,
   'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/lstm11_df_metrics.csv'},
  'resnet': {'f1': 0.8115942028985508,
   'recall': 1.0,
   'precision': 0.6829268292682927,
   'duration': 2.4274494647979736,
   'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/resnet20_df_metr

In [54]:
for device_signal in data:
  bf1 = 0
  best = ''
  for classifier in data[device_signal]:
    # if not data[device_signal][classifier]['recall'] >= 1:
    if bf1 < data[device_signal][classifier]['f1']:
      bf1 = data[device_signal][classifier]['f1']
      best = classifier
  print(data[device_signal][best])

{'f1': 0.8115942028985508, 'recall': 1.0, 'precision': 0.6829268292682927, 'duration': 2.4274494647979736, 'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/resnet20_df_metrics.csv'}
{'f1': 0.7818181818181819, 'recall': 1.0, 'precision': 0.6417910447761194, 'duration': 14.153315782546995, 'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/muse_eeg/test/fcn20_df_metrics.csv'}
{'f1': 0.7878787878787878, 'recall': 0.912280701754386, 'precision': 0.6933333333333334, 'duration': 0.7548892498016357, 'model': '/home/mateus/Documents/school/tcc/code/results_30/END_TO_END/empatica_ppg/test/fcn21_df_metrics.csv'}
{'f1': 0.8062015503875969, 'recall': 0.8813559322033898, 'precision': 0.7428571428571429, 'duration': 0.0166406631469726, 'model': '/home/mateus/Documents/school/tcc/code/results_30/FEATURE_ENGINEERING/empatica_eda/test/xgb16_df_metrics.csv'}


In [ ]:
def latex_all_start(metrics):
  latex = '''\
    \\midrule
\\parbox[t]{{2mm}}{{\\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{\\textbf {{Vol. {volunteer}}}}}}}}} &
      Samsung PPG  & {sms_t}    & {sms_ratio}    & {sms_total}  & {sms_na} & {sms_min} & {sms_max} & {sms_mean} & {sms_std}  \\\\
      & Empatica PPG  & {emp_t}    & {emp_ratio}    & {emp_total}  & {emp_na} & {emp_min} & {emp_max} & {emp_mean} & {emp_std} \\\\
      & Empatica EDA  & {eda_t}    & {eda_ratio}    & {eda_total}  & {eda_na}& {eda_min} & {eda_max} & {eda_mean} & {eda_std}  \\\\
      & Muse EEG  & {eeg_t}    & {eeg_ratio}    & {eeg_total}  & {eeg_na} & N/A & N/A & N/A & N/A \\\\
  '''.format(**metrics)
  print(latex)

In [ ]:

  latex = '''\
    \\midrule
\\parbox[t]{{2mm}}{{\\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{\\textbf {{Vol. {volunteer}}}}}}}}} &
      Samsung PPG  & {sms_t}    & {sms_ratio}    & {sms_total}  & {sms_na} & {sms_min} & {sms_max} & {sms_mean} & {sms_std}  \\\\
      & Empatica PPG  & {emp_t}    & {emp_ratio}    & {emp_total}  & {emp_na} & {emp_min} & {emp_max} & {emp_mean} & {emp_std} \\\\
      & Empatica EDA  & {eda_t}    & {eda_ratio}    & {eda_total}  & {eda_na}& {eda_min} & {eda_max} & {eda_mean} & {eda_std}  \\\\
      & Muse EEG  & {eeg_t}    & {eeg_ratio}    & {eeg_total}  & {eeg_na} & N/A & N/A & N/A & N/A \\\\
  '''.format(**metrics)

samsung ppg
  classifier: cnn
    f1: 0.7482014388489209, best_model: /home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/cnn16_df_metrics.csv
  classifier: fcn
    f1: 0.7883211678832117, best_model: /home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/fcn15_df_metrics.csv
  classifier: lstm
    f1: 0.7567567567567568, best_model: /home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/lstm11_df_metrics.csv
  classifier: resnet
    f1: 0.8115942028985508, best_model: /home/mateus/Documents/school/tcc/code/results_30/END_TO_END/samsung_ppg/test/resnet20_df_metrics.csv
muse eeg
  classifier: cnn
    f1: 0.6666666666666666, best_model: /home/mateus/Documents/school/tcc/code/results_30/END_TO_END/muse_eeg/test/cnn13_df_metrics.csv
  classifier: fcn
    f1: 0.7818181818181819, best_model: /home/mateus/Documents/school/tcc/code/results_30/END_TO_END/muse_eeg/test/fcn20_df_metrics.csv
  classifier: lstm
    f1: 0.323529